# Evolution of $M_{\rm D}$ and $\omega$ over Time

This notebook aims to reproduce Fig. 2 from [Gibson et al. (2017)](https://arxiv.org/abs/1706.04802) which examines how the disc mass $M_{\rm D}$ and the angular frequency $\omega$ evolve over time after `odeint` has solved the ODEs and how the evolution is affected by different values of timescale ratio $\epsilon$ and mass ratio $\delta$. By also examining the fastness parameter evolution, we can observe how the variation in $\epsilon$ and $\delta$ affects the "switch-on" of the propeller regime ($\Omega > 1$).

## 1. Module Imports

Here we import standard libraries `numpy` and `matplotlib` and import `odeint` from `scipy`. Rather than redefine `init_conds` and `ODEs` again, they have been exported to a package called `magnetar` and can also be imported.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from magnetar.funcs import init_conds, ODEs

%matplotlib inline

## 2. Variable Set-up

Here we set up global constants required for the calculation and the values of $\epsilon$ and $\delta$ we'd like to test.

In [ ]:
# Constants
G = 6.674e-8    # Gravitational constant - cgs units
Msol = 1.99e33  # Solar mass - grams
M = 1.4 * Msol  # Magnetar mass - grams
R = 1.0e6       # Magnetar radius - cm
alpha = 0.1     # Sound speed prescription
cs7 = 1.0       # Sound speed - cm/s
GM = G * M

B = 1.0         # Magnetic Field Strength - 10^15 Gauss
P = 5.0         # Initial spin period - milliseconds
MdiscI = 0.001  # Initial disc mass - Solar masses
RdiscI = 100.0  # Disc radius - km

mu = 1.0e15 * B * (R ** 3.0)           # Magnetic Dipole Moment
Rdisc = RdiscI * 1.0e5                 # Convert disc radius to cm
tvisc = Rdisc / (alpha * cs7 * 1.0e7)  # Viscous timescale - seconds

t = np.logspace(0.0, 6.0, base=10.0, num=10001)  # Time array

y = init_conds(P, MdiscI)  # Initial conditions

eps_vals = [1.0, 1.0, 10.0, 10.0]  # Epsilon values
delt_vals = [1.0, 10.0, 1.0, 10.0]  # Delta values

## 3. Solve the Equations and Plot the Results

Here we loop over the desired $\epsilon$ and $\delta$ values and plot the figure.

In [ ]:
# Set up a figure with 3 subplots
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(6,7))
ax3.axhline(1.0, ls='--', c='k')  # Marks the w > 1 condition

colours = ['r', 'r', 'g', 'g']  # Line colours
lines = ['-', '--', '-', '--']  # Linestyles

# Loop over the epsilon and delta values
for i, vals in enumerate(zip(eps_vals, delt_vals)):
    epsilon, delta = vals

    # Solve the ODEs and split the solution
    # n value defaults to 1 when absent from args
    soln = odeint(ODEs, y, t, args=(B, MdiscI, RdiscI, epsilon, delta))
    Mdisc = soln[:,0]
    omega = soln[:,1]
    
    # Radii
    Rm = ((mu ** (4.0 / 7.0)) * (GM ** (-1.0 / 7.0)) * (Mdisc / tvisc) **
          (-2.0 / 7.0))
    Rc = (GM / (omega ** 2.0)) ** (1.0 / 3.0)
    
    # Plotting
    ax1.loglog(t, Mdisc/Msol, ls=lines[i], c=colours[i],
               label='$\epsilon$ = {0}; $\delta$ = {1}'.format(
                      int(epsilon), int(delta)))
    ax2.semilogx(t, omega, ls=lines[i], c=colours[i])
    ax3.semilogx(t, Rm/Rc, ls=lines[i], c=colours[i])

# Plot formatting
ax1.set_xlim(1.0e0, 1.0e6)
ax1.set_ylim(bottom=1.0e-8)
ax2.set_xlim(1.0e0, 1.0e6)
ax3.set_xlim(1.0e0, 1.0e6)
ax3.set_ylim(0.0, 2.0)

ax1.tick_params(axis='both', which='major', labelsize=8)
ax2.tick_params(axis='both', which='major', labelsize=8)
ax3.tick_params(axis='both', which='major', labelsize=8)

ax1.legend(loc='upper right', fontsize=8)
ax3.set_xlabel('Time (s)', fontsize=10)

ax1.set_ylabel('$M_{\\rm D}~\left({\\rm M}_{\odot}\\right)$', fontsize=10)
ax2.set_ylabel('$\omega~\left({\\rm s}^{-1}\\right)$', fontsize=10)
ax3.set_ylabel('$r_{\\rm m}/r_{\\rm c}$', fontsize=10);

<div class="alert alert-block alert-warning">
<b>Warning:</b> This plot is currently not reproducible from the paper due to version issues with python and associated packages.
</div>